# Linking Remote Labor Index to ESCO occupations

Remote Labor Index ([del Rio-Chanona et al. 2020](https://www.oxfordmartin.ox.ac.uk/publications/supply-and-demand-shocks-in-the-covid-19-pandemic-an-industry-and-occupation-perspective/)) indicates what proportion of occupation's work activities can be performed remotely.
RLI is provided for US occupations at the six digit SOC code level - hence, we crosswalk these to the full list of O\*NET occupations, and then do another crosswalk to ESCO.

References:
- Paper by [del Rio-Chanona et al. 2020](https://www.oxfordmartin.ox.ac.uk/publications/supply-and-demand-shocks-in-the-covid-19-pandemic-an-industry-and-occupation-perspective/) where Remote Labor Index was introduced; the table `occupation_variables_and_shock.csv` was accessed and obtained in June 2020 from [here](https://zenodo.org/record/3751068#.X9OxnOmTJTY).
- Crosswalk between US-SOC and O\*NET (`'2010_to_SOC_Crosswalk.xls'`, accessed and obtained in June 2020. Note that in November 2020, O\*NET released an [updated crosswalk](https://www.onetcenter.org/taxonomy.html))

# 0. Import dependencies and inputs

In [1]:
%run ../notebook_preamble.ipy

In [2]:
# Import crosswalk from US-SOC to O*NET
soc10_to_onet = pd.read_excel(data_folder + 'raw/onet/2010_to_SOC_Crosswalk.xls',
              sheet_name='O-NET-SOC 2010 Occupation Listi', skiprows=3)
soc10_to_onet = soc10_to_onet.rename(columns={
    'O*NET-SOC 2010 Code': 'onet_code',
    'O*NET-SOC 2010 Title': 'onet_occupation',
    '2010 SOC Code': 'soc_code',
    '2010 SOC Title': 'soc_title'
})
soc10_to_onet.sample()

,onet_code,onet_occupation,soc_code,soc_title
23,11-3071.01,Transportation Managers,11-3071,"Transportation, Storage, and Distribution Mana..."


In [3]:
# Import crosswalk between ESCO and ONET
esco_onet_xwalk = pd.read_csv(data_folder + 'processed/ESCO_ONET_xwalk_full.csv')

# Import the remote labor index
remote_labor_index = pd.read_csv(data_folder + 'raw/remote_labor_index/occupation_variables_and_shock.csv')

In [4]:
remote_labor_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OCC_CODE                   740 non-null    object 
 1   OCC_TITLE                  740 non-null    object 
 2   remote_labor_index         740 non-null    float64
 3   exposure_infection         644 non-null    float64
 4   essential_score            740 non-null    float64
 5   employment                 740 non-null    float64
 6   median_wage                737 non-null    float64
 7   mean_wage                  737 non-null    float64
 8   supply_shock               740 non-null    float64
 9   demand_shock               740 non-null    float64
 10  total_shock                740 non-null    float64
 11  total_shock_with_positive  740 non-null    float64
dtypes: float64(10), object(2)
memory usage: 69.5+ KB


In [5]:
remote_labor_index.sample(5)

,OCC_CODE,OCC_TITLE,remote_labor_index,exposure_infection,essential_score,employment,median_wage,mean_wage,supply_shock,demand_shock,total_shock,total_shock_with_positive
34,13-1011,"Agents and Business Managers of Artists, Perfo...",0.538462,0.0,0.806460,13970.0,81340.0,86560.0,-0.089326,-0.730601,-0.730601,-0.730601
465,43-5111,"Weighers, Measurers, Checkers, and Samplers, R...",0.294118,1.0,0.704704,61430.0,41110.0,31080.0,-0.208444,-0.125240,-0.208444,-0.208444
692,51-9197,Tire Builders,0.000000,3.0,0.421627,23790.0,36470.0,42230.0,-0.578373,-0.099454,-0.578373,-0.578373
587,49-9081,Wind Turbine Service Technicians,0.000000,5.0,0.987654,4860.0,49190.0,54360.0,-0.012346,-0.034877,-0.034877,-0.034877
674,51-9051,"Furnace, Kiln, Oven, Drier, and Kettle Operato...",0.250000,14.0,0.175732,15700.0,30540.0,37600.0,-0.618201,-0.096433,-0.618201,-0.618201


In [6]:
print(f'Number of unique SOC occupations in "remote_labor_index" dataset: {len(remote_labor_index.OCC_TITLE.unique())}')
print(f'Number of unique SOC occupations in "soc10_to_onet" dataset: {len(soc10_to_onet.soc_title.unique())}')                                                                           

Number of unique SOC occupations in "remote_labor_index" dataset: 740
Number of unique SOC occupations in "soc10_to_onet" dataset: 841


# 1. Link the remote labor index to ESCO occupations


In [7]:
# Link RLI estimates for 6-digit US SOC to 8-digit ONET occupations
# (we're merging using 'OCC_TITLE', as some values in the 'OCC_CODE' column appears to be corrupted)
onet_to_RLI = soc10_to_onet.merge(remote_labor_index[['OCC_TITLE', 'remote_labor_index', 'essential_score', 'total_shock']],
                    right_on='OCC_TITLE',left_on='soc_title', how='left');
onet_to_RLI.head(3)

,onet_code,onet_occupation,soc_code,soc_title,OCC_TITLE,remote_labor_index,essential_score,total_shock
0,11-1011.00,Chief Executives,11-1011,Chief Executives,Chief Executives,0.714286,0.766624,-0.066679
1,11-1011.03,Chief Sustainability Officers,11-1011,Chief Executives,Chief Executives,0.714286,0.766624,-0.066679
2,11-1021.00,General and Operations Managers,11-1021,General and Operations Managers,General and Operations Managers,0.705882,0.607782,-0.115358


In [8]:
onet_to_RLI.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1110 entries, 0 to 1109
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   onet_code           1110 non-null   object 
 1   onet_occupation     1110 non-null   object 
 2   soc_code            1110 non-null   object 
 3   soc_title           1110 non-null   object 
 4   OCC_TITLE           1000 non-null   object 
 5   remote_labor_index  1000 non-null   float64
 6   essential_score     1000 non-null   float64
 7   total_shock         1000 non-null   float64
dtypes: float64(3), object(5)
memory usage: 78.0+ KB


In [9]:
onet_to_RLI[onet_to_RLI.remote_labor_index.isnull()]

,onet_code,onet_occupation,soc_code,soc_title,OCC_TITLE,remote_labor_index,essential_score,total_shock
65,13-1021.00,"Buyers and Purchasing Agents, Farm Products",13-1021,"Buyers and Purchasing Agents, Farm Products",NaN,NaN,NaN,NaN
66,13-1022.00,"Wholesale and Retail Buyers, Except Farm Products",13-1022,"Wholesale and Retail Buyers, Except Farm Products",NaN,NaN,NaN,NaN
67,13-1023.00,"Purchasing Agents, Except Wholesale, Retail, a...",13-1023,"Purchasing Agents, Except Wholesale, Retail, a...",NaN,NaN,NaN,NaN
71,13-1032.00,"Insurance Appraisers, Auto Damage",13-1032,"Insurance Appraisers, Auto Damage",NaN,NaN,NaN,NaN
154,15-2091.00,Mathematical Technicians,15-2091,Mathematical Technicians,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1105,55-3015.00,Command and Control Center Specialists,55-3015,Command and Control Center Specialists,NaN,NaN,NaN,NaN
1106,55-3016.00,Infantry,55-3016,Infantry,NaN,NaN,NaN,NaN
1107,55-3017.00,Radar and Sonar Technicians,55-3017,Radar and Sonar Technicians,NaN,NaN,NaN,NaN
1108,55-3018.00,Special Forces,55-3018,Special Forces,NaN,NaN,NaN,NaN


Note that not all O\*NET occupations have been covered by the crosswalk, and hence for these occupations we will not have Remote Labor Index estimates.

In [17]:
# Link ONET estimates to ESCO
occupations_remote = esco_onet_xwalk.merge(onet_to_RLI[['onet_code', 
                                                        'soc_title', 
                                                        'remote_labor_index', 
                                                        'essential_score',
                                                        'total_shock'
                                                       ]], on='onet_code', how='left')

occupations_remote.sample(3)


,id,concept_uri,preferred_label,isco_level_4,onet_code,onet_occupation,soc_title,remote_labor_index,essential_score,total_shock
954,954,http://data.europa.eu/esco/occupation/4fa660b3...,social work assistant,3412,21-1093.00,social and human service assistants,Social and Human Service Assistants,0.615385,0.989138,-0.004178
2519,2519,http://data.europa.eu/esco/occupation/d6a395eb...,fight director,3435,27-2032.00,choreographers,Choreographers,0.500000,0.725572,-0.219543
842,842,http://data.europa.eu/esco/occupation/479ede81...,journalism lecturer,2310,25-1125.00,"history teachers, postsecondary","History Teachers, Postsecondary",0.850000,1.000000,-0.000000


# 2. Check the Remote Labor Index estimates

In [18]:
occupations_remote.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2942 entries, 0 to 2941
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2942 non-null   int64  
 1   concept_uri         2942 non-null   object 
 2   preferred_label     2942 non-null   object 
 3   isco_level_4        2942 non-null   int64  
 4   onet_code           2921 non-null   object 
 5   onet_occupation     2921 non-null   object 
 6   soc_title           2921 non-null   object 
 7   remote_labor_index  2782 non-null   float64
 8   essential_score     2782 non-null   float64
 9   total_shock         2782 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 252.8+ KB


## 2.1 Null values

In [19]:
# Check which occupations do not have remote labor index estimates
occupations_remote.remote_labor_index.isnull().sum()

160

In [20]:
df = occupations_remote[occupations_remote.remote_labor_index.isnull()].sort_values('isco_level_4')[[
    'preferred_label','onet_code', 'isco_level_4', 'onet_occupation']]
for j, row in df.iterrows():
    print(row.isco_level_4, row.preferred_label)


110 squadron leader
110 artillery officer
110 colonel
110 navy officer
110 air force officer
110 armed forces officer
110 army general
110 fleet commander
110 army major
110 brigadier
110 army captain
110 lieutenant
210 warfare specialist
210 sergeant
210 special forces officer
210 army corporal
310 military engineer
310 intelligence communications interceptor
310 bomb disposal technician
310 infantry soldier
310 air force pilot
2120 gambling, betting, and lottery quality assurance engineer
2131 botanist
2131 toxicologist
2131 botanicals specialist
2131 microbiologist
2131 curator of horticulture
2131 geneticist
2131 biophysicist
2131 biologist
2131 animal feed nutritionist
2131 specialist biomedical scientist
2131 aquaculture biologist
2131 epidemiologist
2131 physiologist
2131 biomedical scientist advanced
2131 biochemist
2359 tutor
2431 advertising media buyer
2659 community artist
3111 metrology technician
3112 energy conservation officer
3113 electromechanical engineering technici

## 2.2 Check the Remote Labor Index estimates for ESCO occupations

Note that the dataset also includes an estimate of whether the occupation could be considered 'essential'. We, however, don't use this variable in the final analyses of the Mapping Career Causeways research report.

In [21]:
# RLI vs 4-digit ISCO-08 code
fig = px.scatter(occupations_remote, y='remote_labor_index', x='isco_level_4',
                 hover_data = ['id','preferred_label','soc_title', 'total_shock'],
                 color_continuous_scale='reds',
                 color='essential_score', title='Remote Labor Index for ESCO occupations')
fig.show()

# 3. Export

In [22]:
# Export the linked data
occupations_remote.to_csv(data_folder + 'processed/linked_data/ESCO_occupations_Remote_Labor_Index.csv', index=False)
